In [ ]:
## python modules used within this notebook
import numpy as np
import matplotlib.pyplot as plt
import happi
from matplotlib.pyplot import *
from matplotlib.colors import LogNorm
import matplotlib.animation as animation

from IPython.display import HTML


plt.rcParams['animation.embed_limit'] = 200.
%matplotlib inline

In [ ]:
cases = ['Landau_Lifshitz','corrected_Landau_Lifshitz', 'Niel', 'Monte_Carlo']
paths = ['../../work_dir/SR/Radiation_'+case for case in cases]

First, we focus on a single simulation

In [ ]:
path = paths[0]
# Time step for the diagnotics
timestep = 6000

In [ ]:
# open Smilei results
S = happi.Open(path, verbose=False)

# Parameters
dt = S.namelist.Main.timestep
dx = S.namelist.Main.cell_length[0]
dy = S.namelist.Main.cell_length[1]
simulation_time = S.namelist.Main.simulation_time

print(' Space steps: %f %f'%(dx,dy))

# ______________________________________________________________________________
# Figure

fig0 = figure()
gs = GridSpec(2, 3)
ax0 = subplot(gs[:,:])

# ______________________________________________________________________________
# Scalar diagnostics

times = S.Scalar("Ukin_electron").getTimes()

# electron kinetic energy
ukin_electron = S.Scalar("Ukin_electron").getData()
ukin_electron_LL = ukin_electron

# Radiated energy wihtout the macro-photons
urad = S.Scalar("Urad").getData()

ax0.plot(times,ukin_electron,color='b',label="Electron",lw=2)
ax0.plot(times,urad,color='purple',label="Radiation",lw=2)

# ______________________________________________________________________________
# Figure properties

t = ax0.set_title('Energy balance')
ax0.set_xlabel(r'$\omega_r t$')
ax0.set_ylabel(r'$\varepsilon / mc^2$')
ax0.set_xlim([200,360])
#ax0.set_yscale('log')

ax0.legend(loc='best', fontsize=15)

fig0.tight_layout()

show()

# fields
# ______________________________________________________________________________
# Figure

fig0 = figure()
gs = GridSpec(9,5)
ax0 = subplot(gs[0:4,:])
ax1 = subplot(gs[4:8,:])

# ______________________________________________________________________________
# Ey electric field

FieldDiag = S.Field(0, "Ey", timesteps=timestep)

f = FieldDiag.getData()[0].T
x = FieldDiag.getAxis("x")
y = FieldDiag.getAxis("y")

im0 = ax0.imshow(
 f, cmap="jet", extent=[x[0],x[-1],y[0],y[-1]], aspect="auto"
)

# ______________________________________________________________________________
# Bz electric field

FieldDiag = S.Field(0, "Bz", timesteps=timestep)

f = FieldDiag.getData()[0].T
x = FieldDiag.getAxis("x")
y = FieldDiag.getAxis("y")

im1 = ax1.imshow(
 f, cmap="jet", extent=[x[0],x[-1],y[0],y[-1]], aspect="auto"
)

# ______________________________________________________________________________
# Figure properties

cb0 = colorbar(im0,format='%g',ax=ax0)
ax0.set_xlabel(r'$\omega_r x / c$')
ax0.set_ylabel(r'$\omega_r y / c$')
ax0.set_ylim([y[0],y[-1]])
ax0.set_xlim([x[0],x[-1]])
cb0.set_label(r'$e E_y / m \omega c$')

cb1 = colorbar(im1,format='%g',ax=ax1)
ax1.set_xlabel(r'$\omega_r x / c$')
ax1.set_ylabel(r'$\omega_r y / c$')
ax1.set_ylim([y[0],y[-1]])
ax1.set_xlim([x[0],x[-1]])
cb1.set_label(r'$e B_z / m \omega$')

#fig0.tight_layout()

show()


# electron density
# ______________________________________________________________________________
# Figure

fig0 = figure()
gs = GridSpec(13,5)
ax0 = subplot(gs[:,:])

# ______________________________________________________________________________
# Electron Diagnostics

PartDiag = S.ParticleBinning(diagNumber=0,timesteps=timestep)

f_density = PartDiag.getData()[0].T
x_density = PartDiag.getAxis("x")
y_density = PartDiag.getAxis("y")

f_density[f_density<=0] = np.nan

im0 = ax0.imshow(
 f_density, cmap="jet", extent=[x_density[0],x_density[-1],y_density[0],y_density[-1]], aspect="auto"
)

# ______________________________________________________________________________
# Figure properties

im0.set_norm(LogNorm())
#im0.set_clim([10.,1e3])
cb0 = colorbar(im0,format='%g',ax=ax0)
ax0.set_xlabel(r'$\omega_r x / c$')
ax0.set_ylabel(r'$\omega_r y / c$')
ax0.set_ylim([y_density[0],y_density[-1]])
cb0.set_label(r'$n_- / n_c$')
t = ax0.set_title(r'Iteration {}'.format(timestep))
t.set_y(1.02)

fig0.tight_layout()

show()


In [ ]:
# Parameters from the simulation
dt = S.namelist.Main.timestep
dx = S.namelist.Main.cell_length[0]
dy = S.namelist.Main.cell_length[1]
simulation_time = S.namelist.Main.simulation_time

PartDiag = S.ParticleBinning(diagNumber=0)

timesteps = PartDiag.getTimesteps()
nt = len(timesteps)

print("Number of iterations: {}".format(nt))

# ______________________________________________________________________________
# Animation function

def animate(i):

    ax.cla()


    # ______________________________________________________________________________
    # Electron Diagnostics

    PartDiag = S.ParticleBinning(diagNumber=0,timesteps=i*500)

    f_density = PartDiag.getData()[0].T
    x_density = PartDiag.getAxis("x")
    y_density = PartDiag.getAxis("y")

    PartDiag = S.ParticleBinning(diagNumber=2,timesteps=i*500)

    f_chi = PartDiag.getData()[0].T
    x_chi = PartDiag.getAxis("x")
    y_chi = PartDiag.getAxis("y")

    f_chi[f_density>0] = f_chi[f_density>0] / f_density[f_density>0]
    print("Iteration {} - Maximal electron quantum parameter: {}".format(timesteps[i],f_chi.max()))
    f_chi[f_density<=0] = np.nan

    im = ax.imshow(
     f_chi, cmap="jet", extent=[x_chi[0],x_chi[-1],y_chi[0],y_chi[-1]], aspect="auto"
    )

    im.set_norm(LogNorm())
    im.set_clim([1e-3,1.])
    ax.set_xlabel(r'$\omega_r x / c$')
    ax.set_ylabel(r'$\omega_r y / c$')
    ax.set_ylim([y_chi[0],y_chi[-1]])
    if i==0:
        cb0 = colorbar(im,format='%g')
        cb0.set_label(r'$\chi_-$')
    t = ax.set_title(r'Iteration {}'.format(i*500))
    t.set_y(1.02)

    return im

# ______________________________________________________________________________
# Figure

fig = figure()
gs = GridSpec(13,5)
ax = subplot(gs[:,:])

animate(0)

fig.tight_layout()

ani = animation.FuncAnimation(fig, animate, np.arange(1, nt),
                              interval=500, blit=False)

plt.close(fig)
HTML(ani.to_jshtml())
# show()

Compare various results

In [ ]:
S_data = dict((cases[k1],  happi.Open(paths[k1], verbose=False)) for k1 in range(len(cases)))

# Create a figure and a 2x2 grid of subplots
fig, axs = plt.subplots(2, 2, figsize=(10, 8))

# Flatten the array of Axes objects
axs = axs.ravel()

for k1 in range(len(cases)):

    # ______________________________________________________________________________
    # Electron Diagnostics
    
    PartDiag = S_data[cases[k1]].ParticleBinning(diagNumber=0,timesteps=timestep)
    
    f_density = PartDiag.getData()[0].T
    x_density = PartDiag.getAxis("x")
    y_density = PartDiag.getAxis("y")
    
    f_density[f_density<=0] = np.nan
    
    im0 = axs[k1].imshow(
     f_density, cmap="jet", extent=[x_density[0],x_density[-1],y_density[0],y_density[-1]], aspect="auto"
    )
    
    # ______________________________________________________________________________
    # Figure properties
    
    im0.set_norm(LogNorm())
    #im0.set_clim([10.,1e3])
    cb0 = colorbar(im0,format='%g',ax=axs[k1])
    axs[k1].set_xlabel(r'$\omega_r x / c$')
    axs[k1].set_ylabel(r'$\omega_r y / c$')
    axs[k1].set_ylim([y_density[0],y_density[-1]])
    cb0.set_label(r'$n_- / n_c$')
    t = axs[k1].set_title(r'Iteration {}'.format(timestep))
    t.set_y(1.02)

# Adjust the spacing between subplots
plt.tight_layout()

# Show the figure
plt.show()

In [ ]:
path = "../../work_dir/SR/Radiation_corrected_Landau_Lifshitz"
S = happi.Open(path, verbose=False)

# Parameters
dt = S.namelist.Main.timestep
dx = S.namelist.Main.cell_length[0]
dy = S.namelist.Main.cell_length[1]
simulation_time = S.namelist.Main.simulation_time

print(' Space steps: %f %f'%(dx,dy))

# ______________________________________________________________________________
# Figure

fig0 = figure()
gs = GridSpec(2, 3)
ax0 = subplot(gs[:,:])

# ______________________________________________________________________________
# Scalar diagnostics

times = S.Scalar("Ukin_electron").getTimes()

# electron kinetic energy
ukin_electron = S.Scalar("Ukin_electron").getData()

# Radiated energy wihtout the macro-photons
urad = S.Scalar("Urad").getData()

ax0.plot(times,ukin_electron,color='b',label="Electron",lw=2)
ax0.plot(times,urad,color='purple',label="Radiation",lw=2)

# ______________________________________________________________________________
# Figure properties

t = ax0.set_title('Energy balance')
ax0.set_xlabel(r'$\omega_r t$')
ax0.set_ylabel(r'$\varepsilon / mc^2$')
ax0.set_xlim([200,360])
#ax0.set_yscale('log')

ax0.legend(loc='best', fontsize=15)

fig0.tight_layout()

show()

In [ ]:
path = "../../work_dir/SR/Radiation_Monte_Carlo"
S = happi.Open(path, verbose=False)

# Parameters
dt = S.namelist.Main.timestep
dx = S.namelist.Main.cell_length[0]
dy = S.namelist.Main.cell_length[1]
simulation_time = S.namelist.Main.simulation_time

print(' Space steps: %f %f'%(dx,dy))

# ______________________________________________________________________________
# Figure

fig0 = figure()
gs = GridSpec(2, 3)
ax0 = subplot(gs[:,:])

# ______________________________________________________________________________
# Scalar diagnostics

times = S.Scalar("Ukin_electron").getTimes()

# electron kinetic energy
ukin_electron = S.Scalar("Ukin_electron").getData()

# Radiated energy wihtout the macro-photons
urad = S.Scalar("Urad").getData()

ax0.plot(times,ukin_electron,color='b',label="Electron",lw=2)
ax0.plot(times,urad,color='purple',label="Radiation",lw=2)
ax0.plot(times,ukin_electron_LL,label="Electron",lw=2)

# ______________________________________________________________________________
# Figure properties

t = ax0.set_title('Energy balance')
ax0.set_xlabel(r'$\omega_r t$')
ax0.set_ylabel(r'$\varepsilon / mc^2$')
ax0.set_xlim([200,360])
#ax0.set_yscale('log')

ax0.legend(loc='best', fontsize=15)

fig0.tight_layout()

show()